# 0_pathway.md

first we convert UCF101 dataset to nframes, basically extracting frames from each video (at 1 fps)

```
/utils/avi2jpg.py
```

[UCF101] --> [UCF101_n_frames]
now [UCF101] has no dependancies

then we write n-frames data for each video folder

```
/utils/n_frames_ucf101.py
```

[UCF101_n_frames] <--->
still has depenedancies

now we have completed Data Loading and have UCF101_n_frames

next we need to apply some basic preprocessing steps on it

```
/dataloaders/ucf_dataset-persistent.py
```

[UCF101_n_frames], [all_videos.txt], [classInd.txt] --> [UCF-preprocessed]
now [UCF101_n_frames] has no dependancies
we have passed it file containing names of all videos rather than one file from ucfTrainTestList
we get UCF-preprocessed

next we apply ROI techiniques on it
we apply a pretrained yolo model to detect people in the dataset and draw the bounding boxes

```
/model-predict-opti_final.py
```

[UCF-preprocessed] --> [UCF_obj_detected]
[UCF-preprocessed] still has dependancies
bboxes are stored in UCF_obj_detected

next we need to alter the generated labels so they represent actions instead of objects
we are also combining the bboxes and adding padding

```
/action_labelling_roi.py 
```

[UCF_obj_detected] --> [UCF_action_labelled_roi]
now [UCF_obj_detected] has no dependancies

# TODO

multi line testing

we get updated labels in UCF_action_labelled_roi

next we need to split the datasets into train test and val

```
/train_test_splitting_final.py
```

[UCF_action_labelled_roi], [UCF-preprocessed] --> [train-data]
now both datasets have no dependancies

it takes labels from "UCF_action_labelled_roi" and images from "UCF-preprocessed"
it splits them
70% train
15% test
15% validation

now we have our training data in /train-data

next we will train a custom yolo model on the training data

```
/train-data/model-train.py
```

[train-data], [ucf101.yaml] --> [best.pt]
now [train-data] has no dependancies, but it does contain model insights
we will have our best.pt model in /train-data/runs

Thats it! we have our custom trained model


# 1_frame_extraction.py

In [ ]:
import os
import subprocess

#frames per second to extract
FPS_RATE = 1

#paths
dir_path = 'Dataset/UCF-101/'
dst_dir_path = 'Dataset/UCF101_n_frames'


for class_name in os.listdir(dir_path):
    class_path = os.path.join(dir_path, class_name)
    if not os.path.isdir(class_path):
        continue

    dst_class_path = os.path.join(dst_dir_path, class_name)
    if not os.path.exists(dst_class_path):
        os.mkdir(dst_class_path)

    for file_name in os.listdir(class_path):
        if '.avi' not in file_name:
            continue
        name, ext = os.path.splitext(file_name)
    
        dst_directory_path = os.path.join(dst_class_path, name)

        video_file_path = os.path.join(class_path, file_name)
        try:
            if os.path.exists(dst_directory_path):
                if not os.path.exists(os.path.join(dst_directory_path, 'image_00001.jpg')):
                    subprocess.call('rm -r \"{}\"'.format(dst_directory_path), shell=True)
                    print('remove {}'.format(dst_directory_path))
                    os.mkdir(dst_directory_path)
            
                else:
                    continue
            else:
                os.mkdir(dst_directory_path)
        except:
            print(dst_directory_path)
            continue
        cmd = 'ffmpeg -i \"{}\" -vf "fps={},scale=-1:240" \"{}/image_%05d.jpg\"'.format(video_file_path, FPS_RATE, dst_directory_path)
    
        print(cmd)
        subprocess.call(cmd, shell=True)
        print('\n')

# 2_writing_frame_data.py

In [ ]:
from __future__ import print_function, division
import os

dir_path = './Dataset/UCF101_n_frames'

for class_name in os.listdir(dir_path):
    class_path = os.path.join(dir_path, class_name)
    if not os.path.isdir(class_path):
        continue

    for file_name in os.listdir(class_path):
        video_dir_path = os.path.join(class_path, file_name)
        image_indices = []
        for image_file_name in os.listdir(video_dir_path):
            if 'image' not in image_file_name:
                continue
            image_indices.append(int(image_file_name[6:11]))

        if len(image_indices) == 0:
            print('no image files', video_dir_path)
            n_frames = 0
        else:
            image_indices.sort(reverse=True)
            n_frames = image_indices[0]
            print(video_dir_path, n_frames)
        with open(os.path.join(video_dir_path, 'n_frames'), 'w') as dst_file:
            dst_file.write(str(n_frames))
        

# 3_data_preprocessing.py

In [ ]:
import os
import pandas as pd
import numpy as np
from torchvision import transforms
import cv2
from torch.utils.data import Dataset, DataLoader
import torch


root_dir = './'
root_list = root_dir + 'Dataset/UCF101_n_frames/'
info_list = root_dir + 'all_videos.txt'
save_dir = root_dir + 'Dataset/UCF-preprocessed'
class_dir = root_dir + 'classInd.txt'

#frames per video
CLIP_LENGTH = 8


class ClipSubstractMean(object):
    def __init__(self, b=104, g=117, r=123):
        self.means = np.array((101.4, 97.7, 90.2))  # B=90.25, G=97.66, R=101.41

    def __call__(self, buffer):
        new_buffer = buffer - self.means
        return new_buffer


class RandomCrop(object):
    """Crop randomly the image in a sample.

    Args:
        output_size (tuple or int): Desired output size. If int, square crop
            is made.
    """

    def __init__(self, output_size=(112, 112)):
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size

    def __call__(self, buffer):
        h, w = buffer.shape[1], buffer.shape[2]
        new_h, new_w = self.output_size

        top = np.random.randint(0, h - new_h)
        left = np.random.randint(0, w - new_w)

        new_buffer = np.zeros((buffer.shape[0], new_h, new_w, 3))
        for i in range(buffer.shape[0]):
            image = buffer[i, :, :, :]
            image = image[top: top + new_h, left: left + new_w]
            new_buffer[i, :, :, :] = image

        return new_buffer


class CenterCrop(object):
    """Crop the image in a sample at the center.

    Args:
        output_size (tuple or int): Desired output size. If int, square crop
            is made.
    """

    def __init__(self, output_size=(112, 112)):
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size

    def __call__(self, buffer):
        h, w = buffer.shape[1], buffer.shape[2]
        new_h, new_w = self.output_size

        top = int(round(h - new_h) / 2.)
        left = int(round(w - new_w) / 2.)

        new_buffer = np.zeros((buffer.shape[0], new_h, new_w, 3))
        for i in range(buffer.shape[0]):
            image = buffer[i, :, :, :]
            image = image[top: top + new_h, left: left + new_w]
            new_buffer[i, :, :, :] = image

        return new_buffer


class RandomHorizontalFlip(object):
    """Horizontally flip the given Images randomly with a given probability.

    Args:
        p (float): probability of the image being flipped. Default value is 0.5
    """

    def __call__(self, buffer):
        if np.random.random() < 0.5:
            for i, frame in enumerate(buffer):
                buffer[i] = cv2.flip(frame, flipCode=1)

        return buffer


class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, buffer):
        # swap color axis because
        # numpy image: batch_size x H x W x C
        # torch image: batch_size x C X H X W
        img = torch.from_numpy(buffer.transpose((3, 0, 1, 2)))

        return img.float().div(255)


class UCFDataset(Dataset):
    r"""A Dataset for a folder of videos. Expects the directory structure to be
    directory->[train/val/test]->[class labels]->[videos]. Initializes with a list
    of all file names, along with an array of labels, with label being automatically
    inferred from the respective folder names.

        Args:
            root_dir (str): path to n_frames_jpg folders.
            info_list (str): path to annotation file.
            split(str): whether create trainset. Default='train'.
            clip_len (int): Determines how many frames are there in each clip. Defaults to 16.

    """

    def __init__(self, class_dir, root_dir, info_list, clip_len=16, save_dir=None):
        self.root_dir = root_dir
        self.clip_len = clip_len
        self.landmarks_frame = pd.read_csv(info_list, delimiter=' ', header=None)
        self.label_class_map = {}
        with open(class_dir) as f:
            for line in f:
                line=line.strip('\n').split(' ')
                self.label_class_map[line[1]] = line[0]
        self.transform = transforms.Compose(
            [ClipSubstractMean(),
                RandomCrop(),
                RandomHorizontalFlip(),
                ToTensor()])
        # The following three parameters are chosen as described in the paper section 4.1
        self.resize_height = 128
        self.resize_width = 171
        self.crop_size = 112

    
        self.save_dir = save_dir
        print(save_dir)
        if self.save_dir is not None:
            os.makedirs(self.save_dir, exist_ok=True)

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, index):
        # Loading and preprocessing.
        video_path = self.landmarks_frame.iloc[index, 0]
        # labels [0,100]
        if self.landmarks_frame.shape[1] == 2:
            labels = self.landmarks_frame.iloc[index, 1] - 1
        else:
            classes = video_path.split('/')[0]
            labels = int(self.label_class_map[classes]) - 1
        buffer = self.get_resized_frames_per_video(video_path)

        if self.transform:
            buffer = self.transform(buffer)

        return buffer, torch.from_numpy(np.array(labels))

    def get_resized_frames_per_video(self, video_path):
        slash_rows = video_path.split('.')
        dir_name = slash_rows[0]
        video_jpgs_path = os.path.join(self.root_dir, dir_name)

        # Read the number of frames from the n_frames file
        data = pd.read_csv(os.path.join(video_jpgs_path, 'n_frames'), delimiter=' ', header=None)
        frame_count = data[0][0]

        # Initialize an array to store all frames
        video_x = np.empty((self.clip_len, self.resize_height, self.resize_width, 3), np.dtype('float32'))
        
        print(video_jpgs_path)

        for i in range(self.clip_len):
            # Compute the frame number based on the clip length
            frame_number = (i * frame_count) // self.clip_len + 1

            # Generate the image filename based on the frame number
            s = "%05d" % frame_number
            image_name = 'image_' + s + '.jpg'
            image_path = os.path.join(video_jpgs_path, image_name)

            if os.path.exists(image_path):
                # Read and resize the image
                tmp_image = cv2.imread(image_path)
                tmp_image = cv2.resize(tmp_image, (self.resize_width, self.resize_height))
                tmp_image = np.array(tmp_image).astype(np.float64)
                tmp_image = tmp_image[:, :, ::-1]  # BGR -> RGB

                # Store the frame in the array
                video_x[i, :, :, :] = tmp_image
            
            if self.save_dir is not None:
                # Save the preprocessed frame to the new directory
                save_image_path = os.path.join(self.save_dir, dir_name)
                
                # Ensure the directory exists, create if necessary
                os.makedirs(save_image_path, exist_ok=True)
                
                # Append the file name to the directory path
                save_image_path = os.path.join(save_image_path, f'image_{i:05d}.jpg')

                
                # print(save_image_path, "saved")
                cv2.imwrite(save_image_path, tmp_image)

        return video_x
    
    

if __name__ == '__main__':
    test_dataloader = DataLoader(
        UCFDataset(
                class_dir=class_dir,
                root_dir=root_list,
                info_list=info_list,
                clip_len=CLIP_LENGTH,
                save_dir=save_dir),
        batch_size=8, shuffle=True, num_workers=0
    )

    for i_batch, (images, targets) in enumerate(test_dataloader):
        print("-----------------Batch: ", i_batch)

        

# 4_object_pose_detection.py

In [ ]:
#performing object detection, pose estimation
from ultralytics import YOLO
import os
import glob

# Define the root directory
root_dir = './Dataset'
processed_dir_name = "/UCF-preprocessed"
output_dir_name = "/UCF_obj_detected"

processed_dir = root_dir + processed_dir_name

# Load a pretrained YOLOv8n model
model = YOLO('yolov8n-pose.pt')

# Walk through all files in the directory
for dirpath, dirnames, filenames in os.walk(processed_dir):
    
    print("dirpath: ", dirpath)
    print("dirnames: ", dirnames)
    print("filenames: ", filenames)
        
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif']
    image_files = [file for file in glob.glob(os.path.join(dirpath, '*')) if os.path.splitext(file)[1].lower() in image_extensions]
    if len(image_files) == 0:
        continue

    # Create the output directory for the current action
    output_dir = dirpath.replace(processed_dir_name, output_dir)
    os.makedirs(output_dir, exist_ok=True)
    

    print("")
    print("")
    model.predict(source=dirpath, conf=0.5, save_txt=True, project=output_dir)
    print("completed model run on ", dirpath)


# 5_action_labelling_roi_processing.py

In [ ]:
import os
"""
has to be run BEFORE train-test-val splitting
because in the splitting, action association is lost
"""

#either ROI or pose estimation
APPLY_ROI = False

# Define the directory paths
root_dir = "./Dataset"
input_dir = root_dir + "/UCF_obj_detected"
output_dir = root_dir + "/UCF_action_labelled_roi"

#%% functions

actions = {
    "ApplyEyeMakeup": 0,
    "ApplyLipstick": 1,
    "Archery": 2,
    "BabyCrawling": 3,
    "BalanceBeam": 4,
    "BandMarching": 5,
    "BaseballPitch": 6,
    "Basketball": 7,
    "BasketballDunk": 8,
    "BenchPress": 9,
    "Biking": 10,
    "Billiards": 11,
    "BlowDryHair": 12,
    "BlowingCandles": 13,
    "BodyWeightSquats": 14,
    "Bowling": 15,
    "BoxingPunchingBag": 16,
    "BoxingSpeedBag": 17,
    "BreastStroke": 18,
    "BrushingTeeth": 19,
    "CleanAndJerk": 20,
    "CliffDiving": 21,
    "CricketBowling": 22,
    "CricketShot": 23,
    "CuttingInKitchen": 24,
    "Diving": 25,
    "Drumming": 26,
    "Fencing": 27,
    "FieldHockeyPenalty": 28,
    "FloorGymnastics": 29,
    "FrisbeeCatch": 30,
    "FrontCrawl": 31,
    "GolfSwing": 32,
    "Haircut": 33,
    "HammerThrow": 34,
    "Hammering": 35,
    "HandstandPushups": 36,
    "HandstandWalking": 37,
    "HeadMassage": 38,
    "HighJump": 39,
    "HorseRace": 40,
    "HorseRiding": 41,
    "HulaHoop": 42,
    "IceDancing": 43,
    "JavelinThrow": 44,
    "JugglingBalls": 45,
    "JumpRope": 46,
    "JumpingJack": 47,
    "Kayaking": 48,
    "Knitting": 49,
    "LongJump": 50,
    "Lunges": 51,
    "MilitaryParade": 52,
    "Mixing": 53,
    "MoppingFloor": 54,
    "Nunchucks": 55,
    "ParallelBars": 56,
    "PizzaTossing": 57,
    "PlayingCello": 58,
    "PlayingDaf": 59,
    "PlayingDhol": 60,
    "PlayingFlute": 61,
    "PlayingGuitar": 62,
    "PlayingPiano": 63,
    "PlayingSitar": 64,
    "PlayingTabla": 65,
    "PlayingViolin": 66,
    "PoleVault": 67,
    "PommelHorse": 68,
    "PullUps": 69,
    "Punch": 70,
    "PushUps": 71,
    "Rafting": 72,
    "RockClimbingIndoor": 73,
    "RopeClimbing": 74,
    "Rowing": 75,
    "SalsaSpin": 76,
    "ShavingBeard": 77,
    "Shotput": 78,
    "SkateBoarding": 79,
    "Skiing": 80,
    "Skijet": 81,
    "SkyDiving": 82,
    "SoccerJuggling": 83,
    "SoccerPenalty": 84,
    "StillRings": 85,
    "SumoWrestling": 86,
    "Surfing": 87,
    "Swing": 88,
    "TableTennisShot": 89,
    "TaiChi": 90,
    "TennisSwing": 91,
    "ThrowDiscus": 92,
    "TrampolineJumping": 93,
    "Typing": 94,
    "UnevenBars": 95,
    "VolleyballSpiking": 96,
    "WalkingWithDog": 97,
    "WallPushups": 98,
    "WritingOnBoard": 99,
    "YoYo": 100
}


# overwrite classes in labels with class corresponding to action_name

# Calculate the ROI for a set of bounding boxes
def calculate_roi(bboxes):
    min_x = float('inf')
    min_y = float('inf')
    max_x = float('-inf')
    max_y = float('-inf')

    roi_bboxes = []

    # Find the minimum and maximum coordinates
    for bbox in bboxes:
        x, y, w, h = bbox
        min_x = min(min_x, x)
        min_y = min(min_y, y)
        max_x = max(max_x, x + w)
        max_y = max(max_y, y + h)

        # Calculate the ROI coordinates
        roi_x = min_x
        roi_y = min_y
        roi_w = max_x - min_x
        roi_h = max_y - min_y

        roi_bboxes.append((roi_x, roi_y, roi_w, roi_h))

    return roi_bboxes

# Add padding to a set of bounding boxes
def add_padding(bboxes, padding_percent = 0.1):
    padded_bboxes = []

    # Calculate the padding values
    padding_x = padding_percent * bboxes[0][2]
    padding_y = padding_percent * bboxes[0][3]

    # Add padding to each bounding box
    for bbox in bboxes:
        x, y, w, h = bbox
        padded_x = x - padding_x
        padded_y = y - padding_y
        padded_w = w + 2 * padding_x
        padded_h = h + 2 * padding_y
        padded_bboxes.append((padded_x, padded_y, padded_w, padded_h))

    return padded_bboxes

#%%
"""
applied on Dataset/UCF_obj_detected
stores in Dataset/UCF_action_labelled

-- labels dir
../Dataset/UCF_obj_detected/{action_name}/{video_name}/predict/labels/{frame_name}.txt

take each of these labels, replace the first number (class id) with class from names[action_name]

"""

#%%

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Iterate over the action directories
for action_name in os.listdir(input_dir):
    action_dir = os.path.join(input_dir, action_name)
    if not os.path.isdir(action_dir):
        continue

    # Iterate over the video directories
    for video_name in os.listdir(action_dir):
        video_dir = os.path.join(action_dir, video_name)
        if not os.path.isdir(video_dir):
            continue

        print(f"Processing {video_name}...")

        # Iterate over the label files
        labels_dir = os.path.join(video_dir, "predict", "labels")
        for frame_name in os.listdir(labels_dir):
            label_file = os.path.join(labels_dir, frame_name)
            if not os.path.isfile(label_file):
                continue

            # Read the label file
            with open(label_file, "r") as f:
                lines = f.readlines()

            # Replace the class id with the action id
            action_id = actions[action_name]
            for i in range(len(lines)):
                class_id, *rest = lines[i].split()
                lines[i] = f"{action_id} {' '.join(rest)}"

            bboxes = [(float(bbox.split()[1]), float(bbox.split()[2]), float(bbox.split()[3]), float(bbox.split()[4])) for bbox in lines]

            if APPLY_ROI:
                roi_bboxes = calculate_roi(bboxes)
                padded_bboxes = add_padding(roi_bboxes)
            else:
                padded_bboxes = add_padding(bboxes)

            # Convert the padded bounding boxes back to lines
            mod_lines = [f"{action_id} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n" for bbox in padded_bboxes]

            # Write the modified label file to the output directory
            output_subdir = os.path.join(output_dir, action_name, video_name, "predict", "labels")
            os.makedirs(output_subdir, exist_ok=True)
            output_file = os.path.join(output_subdir, frame_name)
            with open(output_file, "w") as f:
                f.writelines(mod_lines)



# 6_data_train_test_splitting.py

In [ ]:
"""
1- output format for yolo training

train_dir = /workspaces/video-dataset-preprocess/train-data
^has results

data splitted in 3 folders:
train_dir/train
train_dir/test
train_dir/val

each folder:
./images
./labels

./images takes the images from UCF-preprocessed
./labels takes the labels from UCF_obj_detected

hierarchy of UCF-preprocessed:
./action_name/video_name/images.jpg

hierarchy of UCF_obj_detected:
./action_name/video_name/predict/labels/label.txt


70% training
15% testing
15% validation

"""

import os
import random
import shutil

# Define the paths
root_dir = "./Dataset"
train_dir = root_dir + "/train-data"
ucf_action_labelled_roi_dir_label = root_dir + "/UCF_action_labelled_roi"
ucf_preprocessed_dir_img = root_dir + "/UCF_preprocessed"

# Create train, test, and validation directories
train_data_dir = os.path.join(train_dir, "train")
test_data_dir = os.path.join(train_dir, "test")
val_data_dir = os.path.join(train_dir, "val")

os.makedirs(train_data_dir, exist_ok=True)
os.makedirs(test_data_dir, exist_ok=True)
os.makedirs(val_data_dir, exist_ok=True)



#%% actions images
#####################################################
# images

# Get the list of action names
action_names = os.listdir(ucf_preprocessed_dir_img)

# Iterate over each action name
for action_name in action_names:
    action_dir = os.path.join(ucf_preprocessed_dir_img, action_name)
    video_names = os.listdir(action_dir)

    # Iterate over each video name
    for video_name in video_names:
        video_dir = os.path.join(action_dir, video_name)
        image_files = os.listdir(video_dir)

        # Randomly shuffle the image files
        random.shuffle(image_files)

        # Calculate the number of images for each split
        num_images = len(image_files)
        num_train = int(0.7 * num_images)
        num_test = int(0.15 * num_images)
        num_val = num_images - num_train - num_test

        # Split the image files into train, test, and validation sets
        train_files = image_files[:num_train]
        test_files = image_files[num_train:num_train + num_test]
        val_files = image_files[num_train + num_test:]

        # Create the action directory in train, test, and validation directories
        train_action_dir = os.path.join(train_data_dir, "images")
        test_action_dir = os.path.join(test_data_dir, "images")
        val_action_dir = os.path.join(val_data_dir, "images")

        os.makedirs(train_action_dir, exist_ok=True)
        os.makedirs(test_action_dir, exist_ok=True)
        os.makedirs(val_action_dir, exist_ok=True)

        # Copy the image files to the respective directories
        for file in train_files:
            src = os.path.join(video_dir, file)
            dst = os.path.join(train_action_dir, file)
            shutil.copy(src, dst)

        for file in test_files:
            src = os.path.join(video_dir, file)
            dst = os.path.join(test_action_dir, file)
            shutil.copy(src, dst)

        for file in val_files:
            src = os.path.join(video_dir, file)
            dst = os.path.join(val_action_dir, file)
            shutil.copy(src, dst)


#%%
#####################################################
# labels

# Assign actions to labels
train_label_dir = os.path.join(train_dir, "train")
test_label_dir = os.path.join(train_dir, "test")
val_label_dir = os.path.join(train_dir, "val")

os.makedirs(train_label_dir, exist_ok=True)
os.makedirs(test_label_dir, exist_ok=True)
os.makedirs(val_label_dir, exist_ok=True)


for action_name in action_names:
    action_label_dir = os.path.join(ucf_action_labelled_roi_dir_label, action_name)
    video_names = os.listdir(action_label_dir)

    # Iterate over each video name
    for video_name in video_names:
        labels_dir = os.path.join(action_label_dir, video_name, "predict", "labels")
        label_files = os.listdir(labels_dir)

        # Split the label files into train, test, and validation sets
        train_files = label_files[:num_train]
        test_files = label_files[num_train:num_train + num_test]
        val_files = label_files[num_train + num_test:]

        # Create the action directory in train, test, and validation directories
        train_action_dir = os.path.join(train_label_dir, "labels")
        test_action_dir = os.path.join(test_label_dir, "labels")
        val_action_dir = os.path.join(val_label_dir, "labels")

        os.makedirs(train_action_dir, exist_ok=True)
        os.makedirs(test_action_dir, exist_ok=True)
        os.makedirs(val_action_dir, exist_ok=True)

        # Copy the label files to the respective directories
        for file in train_files:
            src = os.path.join(labels_dir, file)
            dst = os.path.join(train_action_dir, file)
            shutil.copy(src, dst)

        for file in test_files:
            src = os.path.join(labels_dir, file)
            dst = os.path.join(test_action_dir, file)
            shutil.copy(src, dst)

        for file in val_files:
            src = os.path.join(labels_dir, file)
            dst = os.path.join(val_action_dir, file)
            shutil.copy(src, dst)


        

# Print the success message
print("Train-test-val splitting and label assignment completed successfully!")


# 7_action_detection_model_training.py

In [ ]:
#  ACTION DETECTION 
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n-pose.yaml')  # build a new model from YAML
model = YOLO('yolov8n-pose.pt')  # load a pretrained model (recommended for training)
model = YOLO('yolov8n-pose.yaml').load('yolov8n-pose.pt')  # build from YAML and transfer weights

# Train the model
results = model.train(data='action_detection_model_config.yaml', epochs=100, imgsz=640)

